In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import gensim
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.colors as mcolors
from sklearn.manifold import TSNE
from gensim.models import word2vec
import nltk
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/Colab Notebooks/Reviews.csv'

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())



   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [ ]:
# Quick overview of the data
print(data.head())        # Display the first few rows
print(data.info())        # Column types and non-null counts
print(data.describe())    # Summary statistics for numerical columns
print(data.isnull().sum())  # Count of


   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [ ]:
print(data.shape)

(568454, 10)


In [ ]:
# Example: Drop rows with missing values
data = data.dropna()
data


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


**Step 3 Summary: Preprocessing the Text Data**

Text preprocessing is essential to clean and standardise raw text data making it suitable for analysis or machine learning models. The process typically includes:

1. Removing Noise: Removing special characters, punctuation, and numbers.
2. Lowercasing: Converting text to lowercase for uniformity.
3. Tokenisation: Splitting sentences into individual words.
4. Stopword Removal: Eliminating common words like "the" and "is" that don't add much value.
5. Lemmatisation or Stemming: Converting words to their base or root forms (e.g., "running" → "run").

In [ ]:
print(data.columns)


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


In [ ]:
print(data[['Summary', 'Text']].isnull().sum())


Summary    0
Text       0
dtype: int64


In [ ]:
import pandas as pd

# Load the cleaned dataset
#data = pd.read_csv('your_cleaned_dataset.csv')

# Create sentiment labels
def get_sentiment(score):
    if score <= 2:
        return 'Negative'
    elif score == 3:
        return 'Neutral'
    else:
        return 'Positive'

data['Sentiment'] = data['Score'].apply(get_sentiment)

# Check label distribution
print(data['Sentiment'].value_counts())


Sentiment
Positive    443756
Negative     82007
Neutral      42638
Name: count, dtype: int64


In [ ]:
# Splitting data
from sklearn.model_selection import train_test_split

# Features and target
X = data['Text']  # Feature
y = data['Sentiment']  # Target

# Train-validation-test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")


Training set size: 397880
Validation set size: 85260
Test set size: 85261


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')

# Fit and transform on training data
X_train_tfidf = tfidf.fit_transform(X_train)

# Transform validation and test sets
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

print(f"TF-IDF feature shape: {X_train_tfidf.shape}")


TF-IDF feature shape: (397880, 5000)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

# Train the model
model.fit(X_train_tfidf, y_train)

# Validate on the validation set
y_val_pred = model.predict(X_val_tfidf)

# Display validation metrics
print("Validation Performance:")
print(classification_report(y_val, y_val_pred))


Validation Performance:
              precision    recall  f1-score   support

    Negative       0.63      0.74      0.68     12301
     Neutral       0.26      0.63      0.37      6396
    Positive       0.97      0.81      0.88     66563

    accuracy                           0.78     85260
   macro avg       0.62      0.72      0.64     85260
weighted avg       0.87      0.78      0.81     85260



Key Observations:
Class-Specific Metrics:

Negative Sentiment:
Precision: 63% - Fairly good.
Recall: 74% - Indicates the model captures most negative instances.
F1-Score: 68% - Balanced between precision and recall.
Neutral Sentiment:
Precision: 26% - Low; the model struggles to correctly identify neutral instances.
Recall: 63% - It catches more neutral cases but lacks precision.
F1-Score: 37% - Reflects the challenge with this minority class.
Positive Sentiment:
Precision: 97% - Excellent precision for the majority class.
Recall: 81% - Indicates it misses some positive cases.
F1-Score: 88% - Strong overall performance for this dominant class.
Overall Metrics:

Accuracy: 78% - Reasonable but biased toward the majority class (positive sentiment).
Macro Average:
F1-Score: 64% - Accounts for each class equally, showing room for improvement.
Weighted Average:
F1-Score: 81% - Reflects bias toward the dominant positive class.
Challenge with Neutral Class:

Low precision suggests significant misclassification of neutral instances.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization and Padding
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# LSTM Model
model_lstm = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 sentiment classes
])

model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
history_lstm = model_lstm.fit(
    X_train_pad, y_train.map({'Negative': 0, 'Neutral': 1, 'Positive': 2}),
    validation_data=(X_val_pad, y_val.map({'Negative': 0, 'Neutral': 1, 'Positive': 2})),
    epochs=5,
    batch_size=64
)

# Validation Performance
val_loss, val_accuracy = model_lstm.evaluate(X_val_pad, y_val.map({'Negative': 0, 'Neutral': 1, 'Positive': 2}))
print(f"LSTM Validation Accuracy: {val_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 1943s 312ms/step - accuracy: 0.8447 - loss: 0.4291 - val_accuracy: 0.8845 - val_loss: 0.3082
Epoch 2/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 1957s 314ms/step - accuracy: 0.8977 - loss: 0.2790 - val_accuracy: 0.9010 - val_loss: 0.2689
Epoch 3/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 1973s 309ms/step - accuracy: 0.9150 - loss: 0.2353 - val_accuracy: 0.9047 - val_loss: 0.2627
Epoch 4/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 1935s 308ms/step - accuracy: 0.9262 - loss: 0.2048 - val_accuracy: 0.9097 - val_loss: 0.2601
Epoch 5/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 1975s 314ms/step - accuracy: 0.9372 - loss: 0.1758 - val_accuracy: 0.9126 - val_loss: 0.2675
2665/2665 ━━━━━━━━━━━━━━━━━━━━ 110s 41ms/step - accuracy: 0.9110 - loss: 0.2718
LSTM Validation Accuracy: 0.9126


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model for classification (3 labels)
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode_texts(texts, max_len=128):
    return tokenizer(
        list(texts),
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )

train_encodings = encode_texts(X_train)
val_encodings = encode_texts(X_val)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize and Pad Sequences
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=100, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=100, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=100, padding='post')

# Build CNN Model
cnn_model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 sentiment classes
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Model
cnn_model.fit(
    X_train_pad, y_train.map({'Negative': 0, 'Neutral': 1, 'Positive': 2}),
    validation_data=(X_val_pad, y_val.map({'Negative': 0, 'Neutral': 1, 'Positive': 2})),
    epochs=5,
    batch_size=64
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 375s 60ms/step - accuracy: 0.8390 - loss: 0.4539 - val_accuracy: 0.8853 - val_loss: 0.3150
Epoch 2/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 387s 62ms/step - accuracy: 0.8820 - loss: 0.3246 - val_accuracy: 0.8923 - val_loss: 0.2997
Epoch 3/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 428s 60ms/step - accuracy: 0.8930 - loss: 0.2947 - val_accuracy: 0.8998 - val_loss: 0.2822
Epoch 4/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 393s 62ms/step - accuracy: 0.9008 - loss: 0.2726 - val_accuracy: 0.9020 - val_loss: 0.2797
Epoch 5/5
6217/6217 ━━━━━━━━━━━━━━━━━━━━ 436s 61ms/step - accuracy: 0.9084 - loss: 0.2534 - val_accuracy: 0.9045 - val_loss: 0.2762


In [ ]:
# install from PyPI
!pip install openai

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# Train LightGBM Classifier
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train_tfidf, y_train)

# Evaluate on Validation Set
y_val_pred = lgbm_model.predict(X_val_tfidf)
print("Validation Performance:")
print(classification_report(y_val, y_val_pred))

# Evaluate on Test Set
y_test_pred = lgbm_model.predict(X_test_tfidf)
print("Test Performance:")
print(classification_report(y_test, y_test_pred))


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 42.254996 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 814165
[LightGBM] [Info] Number of data points in the train set: 397880, number of used features: 5000
[LightGBM] [Info] Start training from score -1.936019
[LightGBM] [Info] Start training from score -2.590100
[LightGBM] [Info] Start training from score -0.247551


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Validation Performance:
              precision    recall  f1-score   support

    Negative       0.77      0.55      0.64     12301
     Neutral       0.63      0.13      0.21      6396
    Positive       0.87      0.98      0.92     66563

    accuracy                           0.86     85260
   macro avg       0.75      0.55      0.59     85260
weighted avg       0.84      0.86      0.83     85260



/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Test Performance:
              precision    recall  f1-score   support

    Negative       0.77      0.55      0.64     12301
     Neutral       0.64      0.13      0.22      6396
    Positive       0.87      0.98      0.92     66564

    accuracy                           0.85     85261
   macro avg       0.76      0.55      0.59     85261
weighted avg       0.84      0.85      0.83     85261

